In [1]:
import pandas as pd
from os import listdir
import numpy as np

In [2]:
def an_sharpe_ratio(df, N, r):
    # N = numero de barras por año , ejemplo daily N=252, r =1.75% tasa diaria de la reserva federal USD
    X = np.log(df/df.shift(1))
    X = X[1:]
    #############################
    excess_returns = X - (r/N)
    AvgLnRtn = np.mean(excess_returns)
    Std = np.std(excess_returns)
    SharpeRatio = (AvgLnRtn/Std)*np.sqrt(N)
    ###############################
    threshold = 1
    #sharpe_ratio[sharpe_ratio < threshold] = 0
    #sharpe_ratio[sharpe_ratio >= threshold] = 1
    one_sharpe_ratio =SharpeRatio[SharpeRatio >= threshold]

    return one_sharpe_ratio
    #X_daily_rets
    
def change_period(new_time, df):
    tam = new_time
    i=0
    stocks = []
    first = 0
    while i < (len(df)//tam):
        nf = df[first:new_time]
        stock = { 'date':   nf[nf.columns[1]][first],     'timestamp': nf[nf.columns[2]][first],
                  'ticker': nf[nf.columns[3]][first],     'open': nf[nf.columns[4]][first], 
                  'high': nf[nf.columns[5]].max(),        'low': nf[nf.columns[6]].min(),
                  'close': nf[nf.columns[7]][new_time-1], 'volume': nf[nf.columns[8]].sum(),
                  'quantity': nf[nf.columns[9]].sum(),    'tradecount': nf[nf.columns[10]].sum() }
        stocks.append(stock)
    
        first = first + tam
        new_time = new_time + tam
        i = i + 1
    
    if (len(df)%tam) != 0:
        first = len(df)-(len(df)%tam)
        new_time = len(df)
        nf = df[first:new_time]
        stock = { 'date':   nf[nf.columns[1]][first],     'timestamp': nf[nf.columns[2]][first],
                  'ticker': nf[nf.columns[3]][first],     'open': nf[nf.columns[4]][first], 
                  'high': nf[nf.columns[5]].max(),        'low': nf[nf.columns[6]].min(),
                  'close': nf[nf.columns[7]][new_time-1], 'volume': nf[nf.columns[8]].sum(),
                  'quantity': nf[nf.columns[9]].sum(),    'tradecount': nf[nf.columns[10]].sum() }
        stocks.append(stock)
    df_stocks = pd.DataFrame(stocks,columns=['date',     'timestamp', 'ticker', 'open',
                                             'high',     'low',       'close',  'volume', 
                                             'quantity', 'tradecount'])
    #df_stocks.columns()
    return df_stocks

def csv_to_df(ticker):
    file = pd.read_csv('./emisoras/All-Clean-Ticker-Data/'+ticker)
    df = pd.DataFrame(file)
    return df

def limit_df(first, last, df):
    date = df[df.columns[0]]
    df = df[(date>=first_date)&(date<=last_date)]
    return df.reset_index()

def create_df_returns(first_date, last_date, ticker_list, period):
    nf = pd.DataFrame() #df vacio
    df = csv_to_df(ticker_list[0]) #obtenemos el df apartir del csv
    df = limit_df(first_date, last_date, df) #funcion que obtiene un nuevo df apartir de dos fechas y un df
    df = change_period(period, df)
    nf['Date'] = df[df.columns[0]]
    for ticker in ticker_list:
      df = csv_to_df(ticker)
      df = limit_df(first_date, last_date, df)
      df = change_period(period, df)
      nf[ticker] = df[df.columns[6]]
    return nf.reset_index()

def create_df_returns_heder_list(first_date, last_date, header_list, period, ticker):
    nf = pd.DataFrame() #df vacio
    df = csv_to_df(ticker)
    df = limit_df(first_date, last_date, df) #funcion que obtiene un nuevo df apartir de dos fechas y un df
    df = change_period(period, df)
    nf['Timestamp'] = df[df.columns[1]]
    for ticker in ticker_list:
      df = csv_to_df(ticker)
      df = limit_df(first_date, last_date, df)
      df = change_period(period, df)
      nf[ticker] = df[df.columns[6]]
    return nf.reset_index()

#obtener el nombre de los archivos
def ls(pwd): 
    return listdir(pwd)

In [3]:
ticker_list = ls('./emisoras/All-Clean-Ticker-Data/')

first_date = 20140101 #fecha de inicio
last_date = 20140103 #fecha final
period = 15 #periodo
close_df = create_df_returns(first_date, # funcion que regresa un DF con los
                             last_date, #  precios de cierre de las emisoras indicadas
                             ticker_list, 
                             period)    

In [4]:
cl = close_df[close_df.columns[2:]] #tomamos solo los close
sharpe_ratio= df_sharpe_ratio(cl) #con el dataframe de close calculamos los sharpe ratio

AttributeError: 'float' object has no attribute 'log'

In [ ]:
#Obtener los 20 mejores sharpe ratios
final_sharpe_ratios = sharpe_ratio.sort_values( ascending=False)[0:20]

In [ ]:
final_sharpe_ratios